<a href="https://colab.research.google.com/github/jiruneko/IntuitionLLM/blob/main/IntuitionLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

model_id = "microsoft/Phi-3-mini-4k-instruct"

# 利用可能なデバイス判定
device = "cuda" if torch.cuda.is_available() else "cpu"

# Tokenizer 読み込み
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Model 読み込み
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    # GPU があれば自動で割り当て / CPUのみなら None
    device_map="auto" if device == "cuda" else None,
    # GPU 環境なら軽量 dtype / CPU なら float32 のまま
    torch_dtype=(
        torch.bfloat16
        if device == "cuda" and torch.cuda.is_bf16_supported()
        else (torch.float16 if device == "cuda" else torch.float32)
    ),
    trust_remote_code=False,  # 必要なら True に変更可
)

# 推論用パイプライン（任意だけど便利）
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto" if device == "cuda" else None,
)

# 動作確認
out = pipe(
    "Hello, Phi-3!",
    max_new_tokens=50,
    do_sample=True,
    temperature=0.7,
)
print(out[0]["generated_text"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


Hello, Phi-3! I'm interested in exploring your capabilities with a scenario involving a fictional character. Let's imagine we are discussing an intricate plot in a novel. Could you craft a complex character profile for me? The character is named Sir C


In [ ]:
from transformers import pipeline

# パイプラインの作成
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=500,
    do_sample=False
)

Device set to use cuda:0


In [ ]:
# プロンプトを設定
messages = [
    # 鶏に関するおもしろいジョークを作ってください。
    {"role": "user", "content": "鶏に関するおもしろいジョークを作ってください。"}
]

# 出力を生成
output = generator(messages)
print(output[0]["generated_text"])

 鶏のジョーク:

「鶏は、「僕たちは強く、弱く、また強く」という声をついている。」


このジョークは、鶏が声をつくという事実に基づいており、その声が「強く、弱く、また強く」というパターンを持っていることをユーモラスに表現しています。鶏が声をつくことは実際にはありませんが、このジョークは人間のような振る舞いを鶏に対して描写しているため、ユーモアを生み出しています。


In [ ]:
prompt = "悲惨なガーデニングでの災難について、サラに謝罪のメールを書いてください。どうしてそんなことが起きたのか説明してください。"

# 入力プロンプトをトークン化する
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

# テキストの生成
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=20
)

# 出力の表示
print(tokenizer.decode(generation_output[0]))

悲惨なガーデニングでの災難について、サラに謝罪のメールを書いてください。どうしてそんなことが起きたのか説明してください。<|end|>


In [ ]:
print(input_ids)

tensor([[29871,   233,   133,   181,   233,   134,   171, 30371, 30671, 30185,
         30597, 30635, 30203, 30521, 30499, 30199,   234,   132,   192,   236,
           158,   166, 30353, 30773, 30298, 30466, 30330, 30615, 30281, 30353,
           235,   175,   160,   234,   192,   173, 30199, 30604, 30185, 30258,
         30396, 30854, 30298, 30466, 30568, 30955, 30566, 30298, 30267, 31250,
         30465, 30326, 30466, 31110, 30389, 30371, 30589, 30364, 30458, 31558,
         30538, 30366, 30199, 30412,   235,   173,   175, 30592, 30326, 30466,
         30568, 30955, 30566, 30298, 30267]], device='cuda:0')


In [ ]:
for id in input_ids[0]:
  print(tokenizer.decode(id))


�
�
�
�
�
�
な
ガ
ー
デ
ニ
ン
グ
で
の
�
�
�
�
�
�
に
つ
い
て
、
サ
ラ
に
�
�
�
�
�
�
の
メ
ー
ル
を
書
い
て
く
だ
さ
い
。
ど
う
し
て
そ
ん
な
こ
と
が
起
き
た
の
か
�
�
�
明
し
て
く
だ
さ
い
。


In [ ]:
generation_output

tensor([[29871,   233,   133,   181,   233,   134,   171, 30371, 30671, 30185,
         30597, 30635, 30203, 30521, 30499, 30199,   234,   132,   192,   236,
           158,   166, 30353, 30773, 30298, 30466, 30330, 30615, 30281, 30353,
           235,   175,   160,   234,   192,   173, 30199, 30604, 30185, 30258,
         30396, 30854, 30298, 30466, 30568, 30955, 30566, 30298, 30267, 31250,
         30465, 30326, 30466, 31110, 30389, 30371, 30589, 30364, 30458, 31558,
         30538, 30366, 30199, 30412,   235,   173,   175, 30592, 30326, 30466,
         30568, 30955, 30566, 30298, 30267, 32007]], device='cuda:0')

In [ ]:
print(tokenizer.decode(3323))
print(tokenizer.decode(622))
print(tokenizer.decode([3323, 622]))
print(tokenizer.decode(29901))

Sub
ject
Subject
:


In [ ]:
colors_list = [
    "102;194;165", "252;141;98", "141;160;203",
    "231;138;195", "166;216;84", "255;217;47"
]

def show_tokens(sentence, tokenizer_name):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    token_ids = tokenizer(sentence).input_ids
    for idx, t in enumerate(token_ids):
        print(
            f"\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m" +
            tokenizer.decode(t) +
            "\x1b[0m",
            end=" "
        )

In [ ]:
from transformers import AutoModel, AutoTokenizer

# トークナイザーの読み込み
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-xsmall")

# 言語モデルの読み込み
model = AutoModel.from_pretrained("microsoft/deberta-v3-xsmall")

# 文をトークン化
tokens = tokenizer("Hello world", return_tensors="pt")

# トークンを処理
output = model(**tokens)[0]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/241M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

In [ ]:
output.shape

torch.Size([1, 4, 384])

In [ ]:
for token in tokens["input_ids"][0]:
  print(tokenizer.decode(token))

[CLS]
Hello
world
[SEP]


In [ ]:
output

tensor([[[-3.3060, -0.0507, -0.1098,  ..., -0.1704, -0.1618,  0.6932],
         [ 0.8918,  0.0740, -0.1583,  ...,  0.1869,  1.4760,  0.0751],
         [ 0.0871,  0.6364, -0.3050,  ...,  0.4729, -0.1829,  1.0157],
         [-3.1624, -0.1436, -0.0941,  ..., -0.0290, -0.1265,  0.7954]]],
       grad_fn=<NativeLayerNormBackward0>)

In [ ]:
from sentence_transformers import SentenceTransformer

# モデルの読み込み
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

# テキストをテキスト埋め込みに変換
vector = model.encode("Best movie ever!")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
vector.shape

(768,)

In [ ]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 24.9 MB/s eta 0:00:00


In [ ]:
import gensim.downloader as api

model = api.load("glove-wiki-gigaword-50")

[==================================================] 100.0% 66.0/66.0MB downloaded


In [ ]:
model.most_similar([model["king"]], topn=11)

[('king', 1.0000001192092896),
 ('prince', 0.8236179351806641),
 ('queen', 0.7839043140411377),
 ('ii', 0.7746230363845825),
 ('emperor', 0.7736247777938843),
 ('son', 0.766719400882721),
 ('uncle', 0.7627150416374207),
 ('kingdom', 0.7542161345481873),
 ('throne', 0.7539914846420288),
 ('brother', 0.7492411136627197),
 ('ruler', 0.7434253692626953)]